In [ ]:


! pip install mlflow --quiet
! pip install pyngrok --quiet
! pip install torchmetrics

import mlflow
import mlflow.pytorch 
from pyngrok import ngrok

import torchmetrics
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from mlflow.models.signature import infer_signature

import mlflow
import mlflow.pytorch

mlflow.set_tracking_uri('file:///content/drive/MyDrive/Lorusso/BraTS/mlruns')


# Terminate open tunnels if exist
ngrok.kill()
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2Yliuv8VnNyKNcljxgEv6NpZgz8_6ZDBYmEcebUeoX93eGJAE"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri file:///content/drive/MyDrive/Lorusso/BraTS/mlruns --port 5000 & ")



# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=16)

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = SimpleNN()
loss = nn.CrossEntropyLoss()





# Infer the signature of the model
sample_input = X_train_tensor[:1]
model.eval()
with torch.no_grad():
    sample_output = model(sample_input)
signature = infer_signature(sample_input.numpy(), sample_output.numpy())

print("Model signature:", signature)



mlflow.set_experiment("iris_classification_pytorch")

# Start an MLflow run
with mlflow.start_run(run_name = 'iris' ) as run:
    accuracy_metric = torchmetrics.Accuracy(
        task="multiclass", num_classes=3
    )  # Instantiate the Accuracy metric

    for epoch in range(5):  # number of epochs
        total_loss = 0
        total_accuracy = 0

        for inputs, labels in train_loader:
            outputs = model(inputs)
            curr_loss = loss(outputs, labels)
            curr_loss.backward()

            total_loss += curr_loss.item()

            # Calculate accuracy using torchmetrics
            _, preds = torch.max(outputs, 1)
            total_accuracy += accuracy_metric(preds, labels).item()

        avg_loss = total_loss / len(train_loader)
        avg_accuracy = total_accuracy / len(train_loader)

        print(f"Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {avg_accuracy}")
        mlflow.log_metric("loss", avg_loss, step=epoch)
        mlflow.log_metric("accuracy", avg_accuracy, step=epoch)

    # Log the PyTorch model with the signature
    mlflow.pytorch.log_model(model, "model", signature=signature)

    # Log parameters
    mlflow.log_param("epochs", 10)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate", 0.001)

print("Model training and logging complete.")



run_id = mlflow.search_runs(
    experiment_names=["iris_classification_pytorch"],
    filter_string="",
    order_by=["metrics.accuracy DESC"],
).iloc[0].run_id

loaded_model = mlflow.pytorch.load_model(f"runs:/{run_id}/model")
loaded_model 
